In [208]:
import numpy as np
np.random.seed(42)

# Input Collection

## Binary Class

In [209]:
def generate(filename,header=False,append_one=True):
    
    def splitter(line):
        var = line.split()
        x = [float(x) for x in var[:-1]]
        y = int(var[-1])
        return x,y
    
    file = open(filename)
    
    X = []
    Y = []
    
    lines = file.readlines()
    size = len(lines)
    
    if header:
        for i in range(1,size):
            x,y = splitter(lines[i])
            X.append(x)
            Y.append(y)
    else:
        for i in range(0,size):
            x,y = splitter(lines[i])
            X.append(x)
            Y.append(y)
    
    X = np.array(X)
    Y = np.array(Y)
    
    if append_one:
        X = np.insert(X,X.shape[1],values=1,axis=1)
    
    return X,Y

In [210]:
X_train,Y_train = generate('trainLinearlyNonSeparable.txt',header=True)
X_test,Y_test = generate('testLinearlyNonSeparable.txt',header=False)

## Basic Perceptron Algorithm

In [211]:
class BasicPerceptron:
    def __init__(self,low=-10,high=10,max_epoch=1000,lr=0.03):
        self.w = []
        self.low = low
        self.high = high
        self.max_epoch = max_epoch
        self.lr = lr
        pass
    
    def fit(self,X_train,Y_train):
        # setting random weight to weight vector
        self.w = np.random.uniform(self.low,self.high,(X_train.shape[1],1))
        
        for i in range(self.max_epoch):
            
            delta = np.zeros((X_train.shape[1],1))
            
            for instance,target in zip(X_train,Y_train):
                x = instance.reshape(-1,1)
                dot_product = np.dot(self.w.T,x).ravel()[0]
                
                if dot_product < 0 and target == 1: # only here is negative 
                    delta += x
                    continue
                
                if dot_product > 0 and target == 2:
                    delta -= x
                    continue
                
            self.w = self.w + self.lr*delta
        
        return self.w
    
    def predict(self,X,Y):
        correct = 0
        total = len(X)
        
        predicted_class = -1
        for instance,true_class in zip(X,Y):
            x = instance.reshape(-1,1)
            dot_product = np.dot(self.w.T,x).ravel()[0]
            
            if dot_product > 0:
                predicted_class = 1
            if dot_product <0 :
                predicted_class = 2
            
            if predicted_class == true_class:
                correct += 1
        return correct/total

In [212]:
basicPerceptron = BasicPerceptron(low=-5,high=5)
basicPerceptron.fit(X_train,Y_train)
train_accuracy = basicPerceptron.predict(X_train,Y_train)
test_accuracy = basicPerceptron.predict(X_test,Y_test)

train_accuracy,test_accuracy

(0.9775, 0.9725)

## Reward and Punishment Algorithm

In [213]:
class RP_Perceptron:
    def __init__(self,low=-10,high=10,max_epoch=1000,lr=0.03):
        self.w = []
        self.low = low
        self.high = high
        self.max_epoch = max_epoch
        self.lr = lr
        pass
    
    def fit(self,X_train,Y_train):
        # setting random weight to weight vector
        self.w = np.random.uniform(self.low,self.high,(X_train.shape[1],1))
        
        for i in range(self.max_epoch):
            count = 0
            for instance,target in zip(X_train,Y_train):
                x = instance.reshape(-1,1)
                dot_product = np.dot(self.w.T,x).ravel()[0]
                
                if dot_product > 0 and target == 2:
                    count += 1
                    self.w = self.w - self.lr*x
                    
                if dot_product < 0 and target == 1:
                    count += 1
                    self.w = self.w + self.lr*x     
                    
            if count == 0:
                break
        
        return self.w
    
    def predict(self,X,Y):
        correct = 0
        total = len(X)
        
        predicted_class = -1
        for instance,true_class in zip(X,Y):
            x = instance.reshape(-1,1)
            dot_product = np.dot(self.w.T,x).ravel()[0]
            
            if dot_product > 0:
                predicted_class = 1
            if dot_product < 0 :
                predicted_class = 2
            
            if predicted_class == true_class:
                correct += 1
        return correct/total

In [214]:
rp_Perceptron = RP_Perceptron(low=-5,high=5)
rp_Perceptron.fit(X_train,Y_train)
train_accuracy = rp_Perceptron.predict(X_train,Y_train)
test_accuracy = rp_Perceptron.predict(X_test,Y_test)

train_accuracy,test_accuracy

(0.985, 0.975)

## Pocket Algorithm

In [215]:
class Pocket_Perceptron:
    def __init__(self,low=-10,high=10,max_epoch=1000,lr=0.03):
        self.w = []
        self.low = low
        self.high = high
        self.max_epoch = max_epoch
        self.lr = lr
        self.wp = []
        pass
    
    def fit(self,X_train,Y_train):
        # setting random weight to weight vector
        self.w = np.random.uniform(self.low,self.high,(X_train.shape[1],1))
        self.wp = self.w
        misclassification = len(X_train)
        
        for i in range(self.max_epoch):
            count = 0
            
            delta = np.zeros((X_train.shape[1],1))
            
            for instance,target in zip(X_train,Y_train):
                x = instance.reshape(-1,1)
                dot_product = np.dot(self.w.T,x).ravel()[0]
                
                if dot_product < 0 and target == 1: # only here is negative 
                    delta += x
                    count += 1
                    continue
                
                if dot_product > 0 and target == 2:
                    delta -= x
                    count += 1
                    continue
            
            if count < misclassification:
                misclassification = count
                self.wp = self.w
            
            self.w = self.w + self.lr*delta
            
        return self.wp
    
    def predict(self,X,Y):
        correct = 0
        total = len(X)
        
        predicted_class = -1
        for instance,true_class in zip(X,Y):
            x = instance.reshape(-1,1)
            dot_product = np.dot(self.wp.T,x).ravel()[0]
            
            if dot_product >= 0:
                predicted_class = 1
            if dot_product <0 :
                predicted_class = 2
            
            if predicted_class == true_class:
                correct += 1
        return correct/total

In [216]:
pocket_Perceptron = Pocket_Perceptron(low=-5,high=5)
pocket_Perceptron.fit(X_train,Y_train)
train_accuracy = pocket_Perceptron.predict(X_train,Y_train)
test_accuracy = pocket_Perceptron.predict(X_test,Y_test)

train_accuracy,test_accuracy

(0.985, 0.9775)

# Multi Class Perceptron
## Kesler’s Construction

In [217]:
X_train,Y_train = generate('multiclass_train.txt',header=True)
X_test,Y_test = generate('multiclass_test.txt',header=False)

In [218]:
class KeslarConstruction:
    def __init__(self,low=-10,high=10,max_epoch=1000,lr=0.03):
        self.w = []
        self.low = low
        self.high = high
        self.max_epoch = max_epoch
        self.lr = lr
        self.wp = []
        self.number_of_classes = 0
        pass
    
    def fit(self,X_train,Y_train):
        unique_classes = np.unique(Y_train)
        
        self.number_of_classes = len(unique_classes)
        for i in range(self.number_of_classes):
            self.w.append(np.random.uniform(self.low,self.high,(X_train.shape[1],1)))
        self.w = np.array(self.w)
        
        dataset = []
        
        for instance,target in zip(X_train,Y_train):
            current_class = target-1
            for each in range(self.number_of_classes):
                if current_class!= each:
                    instance = instance.reshape(-1,1)
                    row = np.zeros((self.number_of_classes,X_train.shape[1],1))
                    row[current_class] = instance
                    row[each] = -1*instance
                    dataset.append(row)
        
        dataset = np.array(dataset)
        N = len(dataset)
    
        for _ in range(self.max_epoch):
            count = 0
            for i in range(N):
                dot_product = np.dot(self.w.ravel(),dataset[i].ravel())
                if dot_product <= 0:
                    self.w = self.w + self.lr*dataset[i]
                    count += 1
            if count == 0:
                break
    
    def predict(self,X,Y):
        correct = 0
        total = len(X)
        
        for instance,true_class in zip(X,Y):
            max = -10e10
            predicted_class = -1
            for each in range(self.number_of_classes):
                dot_product = np.dot(self.w[each].T,instance.reshape(-1,1)).ravel()
                if dot_product[0] > max:
                    max = dot_product[0]
                    predicted_class = each+1
            
            if true_class == predicted_class:
                correct += 1
                
        return correct/total

In [219]:
kc = KeslarConstruction()
kc.fit(X_train,Y_train)
train_accuracy = kc.predict(X_train,Y_train)
test_accuracy = kc.predict(X_test,Y_test)

print(train_accuracy,test_accuracy)

1.0 0.98
